# Fair Trading Entity Analysis with Phoenix Example
This notebook illustrates:
1. Getting a 'single view' of everything Fair Trading knows about a regulated entity.
2. Understanding relationships between regulated entities (current and historical) to target investigations and detect phoenixing behaviors.
3. An illustrative example of a phoenixing scenario with synthetic data.

Replace example data with real datasets for production use.


## 1. Setup and Imports
Import necessary libraries. Pandas for data manipulation and NetworkX for graph analysis.

In [1]:
# Imports for data handling and graph analysis
import pandas as pd
import networkx as nx

# For date handling
from datetime import datetime


## 2. Identifier Normalization
Define a function to normalize identifiers (ACN/ABN), e.g., remove spaces or standardize formats.

In [2]:
# Function to normalize ACN/ABN by removing spaces
def normalize_id(val):
    """Normalize identifier strings by stripping spaces. Modify if further formatting needed."""
    if isinstance(val, str):
        return val.replace(' ', '')
    return val


## 3. Example DataFrames
Create illustrative example DataFrames for ASIC directors, ABR records, complaints, inspections, and investigations. We also add a synthetic phoenixing example: a director moves from Company A to Company B shortly after cessation.

In [3]:
# Example ASIC directors/officers data including a phoenix scenario
asic_directors_example = pd.DataFrame([
    # Existing example companies
    {'ACN': '123456789', 'CompanyName': 'ABC Pty Ltd', 'DirectorName': 'John Smith', 'AppointmentDate': '2020-05-15', 'CessationDate': None},
    {'ACN': '987654321', 'CompanyName': 'XYZ Holdings Pty Ltd', 'DirectorName': 'Jane Doe', 'AppointmentDate': '2018-03-10', 'CessationDate': '2023-11-30'},
    # Synthetic phoenix example:
    # Company A with director Alice Phoenix ceasing on 2024-01-31
    {'ACN': '111222333', 'CompanyName': 'OldCo Pty Ltd', 'DirectorName': 'Alice Phoenix', 'AppointmentDate': '2019-06-01', 'CessationDate': '2024-01-31'},
    # Company B where the same director is appointed soon after (within 6 months)
    {'ACN': '444555666', 'CompanyName': 'NewCo Pty Ltd', 'DirectorName': 'Alice Phoenix', 'AppointmentDate': '2024-03-15', 'CessationDate': None},
])

# Example ABR data
abr_example = pd.DataFrame([
    {'ABN': '12 345 678 901', 'LegalName': 'Alpha Innovations Pty Ltd', 'BusinessNames': ['Alpha Innovations'], 'TradingNames': ['Alpha Tech'], 'State': 'NSW', 'Postcode': '2000', 'ACN': '123456789', 'GSTStatus': 'Active', 'RegistrationDate': '2019-07-01'},
    {'ABN': '98 765 432 109', 'LegalName': 'Beta Services Pty Ltd', 'BusinessNames': ['Beta Services'], 'TradingNames': ['Beta Consulting'], 'State': 'NSW', 'Postcode': '2140', 'ACN': '987654321', 'GSTStatus': 'Active', 'RegistrationDate': '2021-02-15'},
    # ABR entries for synthetic phoenix example
    {'ABN': '11 122 233 344', 'LegalName': 'OldCo Pty Ltd', 'BusinessNames': ['OldCo'], 'TradingNames': ['OldCo Services'], 'State': 'NSW', 'Postcode': '2100', 'ACN': '111222333', 'GSTStatus': 'Inactive', 'RegistrationDate': '2019-06-01'},
    {'ABN': '44 455 566 677', 'LegalName': 'NewCo Pty Ltd', 'BusinessNames': ['NewCo'], 'TradingNames': ['NewCo Solutions'], 'State': 'NSW', 'Postcode': '2100', 'ACN': '444555666', 'GSTStatus': 'Active', 'RegistrationDate': '2024-03-15'},
])

# Example complaints data
complaints_example = pd.DataFrame([
    {'ComplaintID': 'C202401001', 'BusinessName': 'ABC Pty Ltd', 'ABN': '12 345 678 901', 'DateReceived': '2024-06-01', 'IssueCategory': 'Product quality', 'Status': 'Resolved'},
    {'ComplaintID': 'C202402015', 'BusinessName': 'XYZ Holdings Pty Ltd', 'ABN': '98 765 432 109', 'DateReceived': '2024-05-20', 'IssueCategory': 'Misleading advertising', 'Status': 'Under investigation'},
    # Complaint on OldCo
    {'ComplaintID': 'C202403050', 'BusinessName': 'OldCo Pty Ltd', 'ABN': '11 122 233 344', 'DateReceived': '2024-02-10', 'IssueCategory': 'Non-payment', 'Status': 'Under investigation'},
])

# Example inspections data
inspections_example = pd.DataFrame([
    {'InspectionID': 'I20230101', 'BusinessName': 'ABC Pty Ltd', 'ABN': '12 345 678 901', 'InspectionDate': '2023-10-15', 'Location': 'Sydney', 'Outcome': 'Compliant', 'InspectorName': 'Alice Brown', 'Notes': 'No issues found'},
    {'InspectionID': 'I20230202', 'BusinessName': 'Beta Services Pty Ltd', 'ABN': '98 765 432 109', 'InspectionDate': '2024-03-22', 'Location': 'Parramatta', 'Outcome': 'Non-compliant', 'InspectorName': 'Bob Green', 'Notes': 'Labeling issues'},
    # Inspection on OldCo flagged issues
    {'InspectionID': 'I20240115', 'BusinessName': 'OldCo Pty Ltd', 'ABN': '11 122 233 344', 'InspectionDate': '2024-01-20', 'Location': 'Newcastle', 'Outcome': 'Non-compliant', 'InspectorName': 'Carol White', 'Notes': 'Financial irregularities'},
])

# Example investigations data
investigations_example = pd.DataFrame([
    {'InvestigationID': 'INV2024001', 'BusinessName': 'XYZ Holdings Pty Ltd', 'ABN': '98 765 432 109', 'StartDate': '2024-05-25', 'EndDate': None, 'Outcome': 'Ongoing', 'RelatedEntities': ['Alpha Innovations Pty Ltd']},
    {'InvestigationID': 'INV2023005', 'BusinessName': 'Gamma Trading Pty Ltd', 'ABN': '11 222 333 444', 'StartDate': '2023-01-10', 'EndDate': '2023-12-05', 'Outcome': 'Enforcement action taken', 'RelatedEntities': []},
    # Investigation on OldCo before phoenix
    {'InvestigationID': 'INV2024010', 'BusinessName': 'OldCo Pty Ltd', 'ABN': '11 122 233 344', 'StartDate': '2024-02-15', 'EndDate': None, 'Outcome': 'Ongoing', 'RelatedEntities': []},
])

# Normalize IDs in example DataFrames
for df, col in [(asic_directors_example, 'ACN'), (abr_example, 'ACN'), (abr_example, 'ABN'),
                (complaints_example, 'ABN'), (inspections_example, 'ABN'), (investigations_example, 'ABN')]:
    df[col] = df[col].apply(normalize_id)

# Display examples (optional) to verify
print('ASIC directors example:'); display(asic_directors_example)
print('ABR example:'); display(abr_example)


ASIC directors example:


,ACN,CompanyName,DirectorName,AppointmentDate,CessationDate
0,123456789,ABC Pty Ltd,John Smith,2020-05-15,None
1,987654321,XYZ Holdings Pty Ltd,Jane Doe,2018-03-10,2023-11-30
2,111222333,OldCo Pty Ltd,Alice Phoenix,2019-06-01,2024-01-31
3,444555666,NewCo Pty Ltd,Alice Phoenix,2024-03-15,None


ABR example:


,ABN,LegalName,BusinessNames,TradingNames,State,Postcode,ACN,GSTStatus,RegistrationDate
0,12345678901,Alpha Innovations Pty Ltd,[Alpha Innovations],[Alpha Tech],NSW,2000,123456789,Active,2019-07-01
1,98765432109,Beta Services Pty Ltd,[Beta Services],[Beta Consulting],NSW,2140,987654321,Active,2021-02-15
2,11122233344,OldCo Pty Ltd,[OldCo],[OldCo Services],NSW,2100,111222333,Inactive,2019-06-01
3,44455566677,NewCo Pty Ltd,[NewCo],[NewCo Solutions],NSW,2100,444555666,Active,2024-03-15


## 4. Single View Function
Define and demonstrate a function to aggregate all known data about an entity.

In [11]:
# Single view function aggregates data by ACN or ABN
def get_single_view(acn=None, abn=None):
    """Return aggregated info for an entity by ACN or ABN."""
    result = {}
    # If ACN provided: fetch ASIC and ABR info
    if acn:
        acn_norm = normalize_id(acn)
        # ASIC: get first company name
        df_dir = asic_directors_example[asic_directors_example['ACN'] == acn_norm]
        if not df_dir.empty:
            result['CompanyName_ASIC'] = df_dir.iloc[0]['CompanyName']
        # ABR: fetch legal and trading names
        df_abr = abr_example[abr_example['ACN'] == acn_norm]
        if not df_abr.empty:
            abr_row = df_abr.iloc[0]
            result.update({
                'LegalName_ABR': abr_row['LegalName'],
                'BusinessNames': abr_row['BusinessNames'],
                'TradingNames': abr_row['TradingNames'],
                'ABN': abr_row['ABN'],
                'State': abr_row['State'],
                'Postcode': abr_row['Postcode'],
                'GSTStatus': abr_row['GSTStatus'],
                'RegistrationDate': abr_row['RegistrationDate']
            })
        else:
            result['ABR_record'] = 'Not found'
    # If ABN provided: fetch ABR and related datasets
    if abn:
        abn_norm = normalize_id(abn)
        df_abr2 = abr_example[abr_example['ABN'] == abn_norm]
        if not df_abr2.empty:
            abr_row2 = df_abr2.iloc[0]
            result.setdefault('LegalName_ABR', abr_row2['LegalName'])
            result.setdefault('BusinessNames', abr_row2['BusinessNames'])
            result.setdefault('TradingNames', abr_row2['TradingNames'])
            result.setdefault('ACN', abr_row2['ACN'])
            result.setdefault('State', abr_row2['State'])
            result.setdefault('Postcode', abr_row2['Postcode'])
            result.setdefault('GSTStatus', abr_row2['GSTStatus'])
            result.setdefault('RegistrationDate', abr_row2['RegistrationDate'])
        else:
            result['ABR_record'] = 'Not found'
        # Complaints
        df_com = complaints_example[complaints_example['ABN'] == abn_norm]
        result['Complaints'] = df_com.to_dict(orient='records') if not df_com.empty else []
        # Inspections
        df_ins = inspections_example[inspections_example['ABN'] == abn_norm]
        result['Inspections'] = df_ins.to_dict(orient='records') if not df_ins.empty else []
        # Investigations
        df_inv = investigations_example[investigations_example['ABN'] == abn_norm]
        result['Investigations'] = df_inv.to_dict(orient='records') if not df_inv.empty else []
    # Directors if ACN known
    if 'ACN' in result:
        df_dir2 = asic_directors_example[asic_directors_example['ACN'] == normalize_id(result['ACN'])]
        result['Directors'] = df_dir2.to_dict(orient='records') if not df_dir2.empty else []
    return result


### Demonstrate Single View
Example: get single view for OldCo (synthetic phoenix case) and NewCo.

In [12]:
# Single view for OldCo via ABN
sv_oldco = get_single_view(abn='11 122 233 344')
print('Single view for OldCo Pty Ltd:')
sv_oldco

# Single view for NewCo via ACN
sv_newco = get_single_view(acn='444555666')
print('Single view for NewCo Pty Ltd:')
sv_newco


Single view for OldCo Pty Ltd:
Single view for NewCo Pty Ltd:


{'CompanyName_ASIC': 'NewCo Pty Ltd',
 'LegalName_ABR': 'NewCo Pty Ltd',
 'BusinessNames': ['NewCo'],
 'TradingNames': ['NewCo Solutions'],
 'ABN': '44455566677',
 'State': 'NSW',
 'Postcode': '2100',
 'GSTStatus': 'Active',
 'RegistrationDate': '2024-03-15'}

## 5. Relationship Graph
Build a graph of companies and directors to analyze shared relationships.

In [13]:
# Build relationship graph: nodes are companies (ACN) and persons (DirectorName)
def build_relationship_graph():
    G = nx.Graph()
    for _, row in asic_directors_example.iterrows():
        comp_node = f"ACN:{normalize_id(row['ACN'])}"
        person_node = f"Person:{row['DirectorName']}"
        # Add nodes with type metadata
        G.add_node(comp_node, type='company')
        G.add_node(person_node, type='person')
        # Add edge with appointment and cessation dates for temporal analysis
        G.add_edge(comp_node, person_node,
                   relation='director',
                   appointment=row['AppointmentDate'],
                   cessation=row['CessationDate'])
    return G


### Finding Related Entities
Define a function to find companies sharing directors with a given ACN.

In [14]:
def find_related_entities(graph, acn):
    """Return ACNs of companies related via shared directors."""
    node = f"ACN:{normalize_id(acn)}"
    related = set()
    if not graph.has_node(node):
        return related
    # For each director connected to the company
    for person in graph.neighbors(node):
        # For each other company that director is connected to
        for nbr in graph.neighbors(person):
            if nbr.startswith('ACN:') and nbr != node:
                related.add(nbr.replace('ACN:', ''))
    return related


### Demonstrate Related Entities
Check related companies for OldCo and NewCo in the phoenix example.

In [8]:
G = build_relationship_graph()
related_oldco = find_related_entities(G, acn='111222333')
print('Companies related to OldCo (shared directors):', related_oldco)
related_newco = find_related_entities(G, acn='444555666')
print('Companies related to NewCo (shared directors):', related_newco)


Companies related to OldCo (shared directors): {'444555666'}
Companies related to NewCo (shared directors): {'111222333'}


## 6. Phoenix Pattern Detection
Define and demonstrate detection of phoenix-like behavior based on director movements.

In [15]:
# Function to detect phoenix candidates based on director appointment/cessation
def detect_phoenix_candidates(asic_df, window_days=180):
    """Identify director movements suggesting phoenix behavior within window_days."""
    df = asic_df.copy()
    # Parse date columns
    df['AppointmentDate'] = pd.to_datetime(df['AppointmentDate'])
    df['CessationDate'] = pd.to_datetime(df['CessationDate'], errors='coerce')
    candidates = []
    # Group by director to examine their company movements
    for person, group in df.groupby('DirectorName'):
        group_sorted = group.sort_values('AppointmentDate')
        # For each past company appointment
        for i, row_a in group_sorted.iterrows():
            if pd.isna(row_a['CessationDate']):
                # If still active, skip as a potential old company
                continue
            cease_date = row_a['CessationDate']
            # Look for later appointments after cessation
            later = group_sorted[group_sorted['AppointmentDate'] > cease_date]
            for j, row_b in later.iterrows():
                # If appointment within window_days after cessation
                if (row_b['AppointmentDate'] - cease_date).days <= window_days:
                    candidates.append({
                        'Director': person,
                        'OldACN': row_a['ACN'],
                        'NewACN': row_b['ACN'],
                        'CessationDate': cease_date.date(),
                        'NewAppointmentDate': row_b['AppointmentDate'].date()
                    })
    return pd.DataFrame(candidates)


### Demonstrate Phoenix Detection
Run detection on the example data to identify the synthetic phoenix scenario.

In [10]:
# Detect phoenix candidates in example data
phoenix_df = detect_phoenix_candidates(asic_directors_example)
print('Phoenix candidates identified:')
phoenix_df


Phoenix candidates identified:


,Director,OldACN,NewACN,CessationDate,NewAppointmentDate
0,Alice Phoenix,111222333,444555666,2024-01-31,2024-03-15


## 7. Interpretation of Results
- The `phoenix_df` should show Alice Phoenix moving from OldCo Pty Ltd (ACN 111222333) to NewCo Pty Ltd (ACN 444555666) within the specified window.
- The relationship graph will show that OldCo and NewCo share the director Alice Phoenix.

Investigators can use these outputs to:
- Review the single views for both entities (e.g., OldCo had non-compliant inspections and ongoing investigations; NewCo is newly registered under the same director).
- Examine network links and temporal patterns for further risk scoring.


## 8. Next Steps for Production
1. Replace example DataFrames with data loaded from actual databases or APIs.
2. Enhance `get_single_view` to include summary metrics (e.g., counts, last action dates, risk flags).
3. Expand graph: add additional edges for shareholdings, common addresses, investigations linking multiple entities.
4. Store graph in a scalable graph database (e.g., Neo4j) for real-time queries and visualization.
5. Integrate alerting: schedule regular checks for new phoenix patterns or clusters of concern.
6. Provide an interactive dashboard for Intelligence, Compliance & Investigations teams.
